In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 538 kB/s 
     |████████████████████████████████| 465 kB 59.1 MB/s 


In [290]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
import statistics as stats
import re

from konlpy.tag import Okt
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [272]:
songs = pd.read_excel('/content/song_lyrics_labeled.xlsx')
songs.dropna(inplace=True)
songs

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,year,singer,names,categories,lyrics,hope,money,...,friend,equal,swear,nature,regret,memory,sf,food,confi,enjoy
0,0,0,0,1964,금호동,내일또만납시다,희망,하루의 일을 끝내고 돌아가는거리엔 사람의 물결하늘엔 별이 하나 둘 반짝이면가로등 하...,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1964,최숙자,눈물의연평도,"후회, 자연",조기를 담뿍잡아기폭을 올리고온다던 그배는어이하여 아니오나수평선 바라보며그이름 부르면...,0,0,...,0,0,0,1,1,0,0,0,0,0
2,2,2,2,1964,이미자,님이라부르리까,사랑,<1절 임이라 부르리까 당신이라고 부르리까 사랑을 하면서도 사랑을 참고사는 마음으로...,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,3,1964,이미자,동백아가씨,"사랑, 후회",헤일 수 없이 수많은 밤을내 가슴 도려내는 아픔에 겨워얼마나 울었던가 동백아가씨그리...,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,4,4,1964,현미,떠날때는말없이,"사랑, 후회",그 날 밤 그 자리에둘이서 만났을 때똑같은 그 순간에똑같은 마음이달빛에 젖은 채밤새...,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6195,6195,6195,7432,2021,IVE(아이브),ELEVEN,사랑,따분한 나의 눈빛이 무표정했던 얼굴이 널 보며 빛나고 있어 널 담은 눈동자는 내 ...,0,0,...,0,0,0,0,0,0,0,0,0,0
6196,6196,6196,7433,2021,다비치,매일크리스마스(EverydayChristmas),사랑,모두가 기다려온 이 겨울 두 손엔 가득 선물을 행복한 사람들 빛...,0,0,...,0,0,0,0,0,0,0,0,0,0
6197,6197,6197,7434,2021,아이유(IU),겨울잠,희망,때 이른 봄 몇 송이 꺾어다 너의 방 문 앞에 두었어 긴 잠 실컷 자고 나오면 그때...,1,0,...,0,0,0,0,0,0,0,0,0,0
6198,6198,6198,7435,2021,폴킴(PaulKim),찬란한계절,사랑,비가 내리는 반대편으로 여름을 향해 달리네 내게로 내미는 어떤 것도 끝내 붙잡지 못...,0,0,...,0,0,0,0,0,0,0,0,0,0


In [273]:
X = songs.lyrics
y = songs.love

In [276]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)

In [277]:
okt = Okt()

In [278]:
def clean_lyrics(df):

  lst = ['Noun', 'Adjective', 'Adverb', 'Verb']
  final = []
  
  for i in tqdm(range(df.shape[0])):
    result = []
    test_txt = re.sub("[^가-힣\\s]", "", df['lyrics'].iloc[i])
    test_review = okt.pos(test_txt, stem=True)
    for j in range(len(test_review)):
      if test_review[j][1] in lst:
        result.append(test_review[j][0])
    final.append(result)
  
  return final

In [279]:
def stopword_list(df):

  lst = ['Noun', 'Adjective', 'Adverb', 'Verb']
  final = []
  
  for i in tqdm(range(df.shape[0])):
    test_txt = re.sub("[^가-힣\\s]", "", df['lyrics'].iloc[i])
    test_review = okt.pos(test_txt, stem=True)
    for j in range(len(test_review)):
      if test_review[j][1] in lst:
        final.append(test_review[j][0])

  num_counts = Counter(final)
  sorted_counts = sorted(num_counts.items(), key=lambda x:x[1])

  return sorted_counts

In [280]:
def find_stopwords(lst):
  # p1 < p2
  p1=0.8
  p2=0.9
  idx1 = int(p1*len(lst))
  idx2 = int(p2*len(lst))
  return lst[idx1:idx2]

In [281]:
# count는 각 단어별로 개수
count = stopword_list(songs)

  0%|          | 0/6200 [00:00<?, ?it/s]

In [282]:
count

[('조기', 1),
 ('천간', 1),
 ('헤일', 1),
 ('러러', 1),
 ('밀짚모자', 1),
 ('음음음음', 1),
 ('두꺼비', 1),
 ('고양', 1),
 ('아랫목', 1),
 ('쓰르라미', 1),
 ('송아진', 1),
 ('잔칫날', 1),
 ('떡방아', 1),
 ('물바', 1),
 ('야속트', 1),
 ('무정트', 1),
 ('테프', 1),
 ('고국', 1),
 ('솟네뭉', 1),
 ('언덕길', 1),
 ('쓸쓸', 1),
 ('뿜는', 1),
 ('배귀', 1),
 ('없네깨스등', 1),
 ('도회지', 1),
 ('거석', 1),
 ('헤여질', 1),
 ('심정은', 1),
 ('일반', 1),
 ('야멸차', 1),
 ('움막', 1),
 ('짊어', 1),
 ('항라', 1),
 ('왕거미', 1),
 ('성황', 1),
 ('도토리', 1),
 ('묵', 1),
 ('허리춤', 1),
 ('금봉', 1),
 ('금잔디', 1),
 ('노랭', 1),
 ('중세', 1),
 ('산통', 1),
 ('사이공', 1),
 ('업힌', 1),
 ('마즈', 1),
 ('순풍', 1),
 ('백사', 1),
 ('눈믈뿐', 1),
 ('아타', 1),
 ('밤마', 1),
 ('타관', 1),
 ('영도다리', 1),
 ('지향', 1),
 ('옹개종개', 1),
 ('풍기면', 1),
 ('로케트', 1),
 ('멕이', 1),
 ('아주까리', 1),
 ('몽매', 1),
 ('조롱조롱', 1),
 ('정성만', 1),
 ('워싱턴', 1),
 ('조만', 1),
 ('괄', 1),
 ('불끈쥐', 1),
 ('무자비하다', 1),
 ('더냐', 1),
 ('옥답', 1),
 ('부들', 1),
 ('래일', 1),
 ('카나리아', 1),
 ('배배새', 1),
 ('기숙사', 1),
 ('리피', 1),
 ('예외', 1),
 ('도살', 1),
 ('다시피', 1),
 ('사노라면', 1),
 ('

In [283]:
dic_list = []
# 노래 가사에 나온 단어들을 정리한 리스트를 만들어주자
for i in range(len(count)):
  dic_list.append(count[i][0])

dic_list

['조기',
 '천간',
 '헤일',
 '러러',
 '밀짚모자',
 '음음음음',
 '두꺼비',
 '고양',
 '아랫목',
 '쓰르라미',
 '송아진',
 '잔칫날',
 '떡방아',
 '물바',
 '야속트',
 '무정트',
 '테프',
 '고국',
 '솟네뭉',
 '언덕길',
 '쓸쓸',
 '뿜는',
 '배귀',
 '없네깨스등',
 '도회지',
 '거석',
 '헤여질',
 '심정은',
 '일반',
 '야멸차',
 '움막',
 '짊어',
 '항라',
 '왕거미',
 '성황',
 '도토리',
 '묵',
 '허리춤',
 '금봉',
 '금잔디',
 '노랭',
 '중세',
 '산통',
 '사이공',
 '업힌',
 '마즈',
 '순풍',
 '백사',
 '눈믈뿐',
 '아타',
 '밤마',
 '타관',
 '영도다리',
 '지향',
 '옹개종개',
 '풍기면',
 '로케트',
 '멕이',
 '아주까리',
 '몽매',
 '조롱조롱',
 '정성만',
 '워싱턴',
 '조만',
 '괄',
 '불끈쥐',
 '무자비하다',
 '더냐',
 '옥답',
 '부들',
 '래일',
 '카나리아',
 '배배새',
 '기숙사',
 '리피',
 '예외',
 '도살',
 '다시피',
 '사노라면',
 '극성',
 '요게바',
 '생글생글',
 '대풍',
 '계시',
 '뭄',
 '부림',
 '주안상',
 '투술',
 '홀랑',
 '벙글',
 '급제',
 '뭉게뭉게',
 '줏',
 '높다랗',
 '계급',
 '넌즛',
 '염불',
 '여승',
 '나침판',
 '철선',
 '상륙',
 '주마',
 '금테',
 '온천장',
 '아랴바람',
 '아랴가슴',
 '룩',
 '골려',
 '나자빠지다',
 '전차',
 '눈총',
 '다재',
 '쩌구',
 '간주곡',
 '작성자',
 '최영식',
 '보데',
 '자해',
 '뚱뚱보',
 '여드름',
 '잡새',
 '봉황',
 '문전',
 '구곡',
 '무인',
 '수립',
 '비조',
 '뭍새',
 '명랑하다',
 '꾀꾀',
 '섬색시',
 '낙엽송',
 '이름뚜

In [284]:
# 각 단어를 key 로 설정하고 value 는 모두 0으로 지정해주자
dictionary = dict.fromkeys(dic_list, 0)

dictionary

{'조기': 0,
 '천간': 0,
 '헤일': 0,
 '러러': 0,
 '밀짚모자': 0,
 '음음음음': 0,
 '두꺼비': 0,
 '고양': 0,
 '아랫목': 0,
 '쓰르라미': 0,
 '송아진': 0,
 '잔칫날': 0,
 '떡방아': 0,
 '물바': 0,
 '야속트': 0,
 '무정트': 0,
 '테프': 0,
 '고국': 0,
 '솟네뭉': 0,
 '언덕길': 0,
 '쓸쓸': 0,
 '뿜는': 0,
 '배귀': 0,
 '없네깨스등': 0,
 '도회지': 0,
 '거석': 0,
 '헤여질': 0,
 '심정은': 0,
 '일반': 0,
 '야멸차': 0,
 '움막': 0,
 '짊어': 0,
 '항라': 0,
 '왕거미': 0,
 '성황': 0,
 '도토리': 0,
 '묵': 0,
 '허리춤': 0,
 '금봉': 0,
 '금잔디': 0,
 '노랭': 0,
 '중세': 0,
 '산통': 0,
 '사이공': 0,
 '업힌': 0,
 '마즈': 0,
 '순풍': 0,
 '백사': 0,
 '눈믈뿐': 0,
 '아타': 0,
 '밤마': 0,
 '타관': 0,
 '영도다리': 0,
 '지향': 0,
 '옹개종개': 0,
 '풍기면': 0,
 '로케트': 0,
 '멕이': 0,
 '아주까리': 0,
 '몽매': 0,
 '조롱조롱': 0,
 '정성만': 0,
 '워싱턴': 0,
 '조만': 0,
 '괄': 0,
 '불끈쥐': 0,
 '무자비하다': 0,
 '더냐': 0,
 '옥답': 0,
 '부들': 0,
 '래일': 0,
 '카나리아': 0,
 '배배새': 0,
 '기숙사': 0,
 '리피': 0,
 '예외': 0,
 '도살': 0,
 '다시피': 0,
 '사노라면': 0,
 '극성': 0,
 '요게바': 0,
 '생글생글': 0,
 '대풍': 0,
 '계시': 0,
 '뭄': 0,
 '부림': 0,
 '주안상': 0,
 '투술': 0,
 '홀랑': 0,
 '벙글': 0,
 '급제': 0,
 '뭉게뭉게': 0,
 '줏': 0,
 '높다랗': 0,
 '계급':

In [285]:
lyrics_list = clean_lyrics(songs)

  0%|          | 0/6200 [00:00<?, ?it/s]

In [286]:
def doc_frequency(lyrics):

  for lyric in tqdm(lyrics):
    num_count = Counter(lyric)
    for word in dic_list:
      if num_count[word] > 0:
        dictionary[word] += 1

In [287]:
doc_frequency(lyrics_list)

  0%|          | 0/6200 [00:00<?, ?it/s]

In [ ]:
def find_frequency():
  

In [133]:
# a = find_stopwords(count)

In [ ]:
# df = pd.DataFrame(a)
# csv = df.to_csv('stopwords.csv', index = False)
# pd.read_csv('stopwords.csv')

In [288]:
def preprocessing(review): 
    okt = Okt()
    
    # f = open('nsmc_stopwords.txt')
    # stop_words = f.read().split()
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
    word_review = okt.pos(review_text, stem=True)
    
    # 노이즈 & 불용어 제거
    # word_review = [(token, pos) for token, pos in word_review if not token in stop_words]
    
    # 명사, 동사, 형용사 추출
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']]

    return word_review

In [291]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

bow = CountVectorizer(tokenizer=preprocessing, min_df=5, max_df=0.5) 

X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

KeyboardInterrupt: ignored

In [292]:
tfidf = TfidfVectorizer(tokenizer=preprocessing, max_features=2000, min_df=5, max_df=0.5) 

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [293]:
# 모델 1 : Logistic Regression 모형
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
log_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(log_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(log_clf.score(X_test_tfidf, y_test)))

Train set score: 0.884
Test set score: 0.845
